# Beginner Capstone Project: Enhanced Number Guessing Game

**Duration**: 2-3 hours  
**Stage**: Beginner Capstone

---

## 🎯 Project Overview

Create an enhanced number guessing game that demonstrates all the concepts you've learned in the beginner stage:
- Variables and mutability
- Functions and control flow
- Ownership and borrowing
- Basic pattern matching
- Error handling
- User interaction

## 🎮 Game Features

Your enhanced game will include:
1. **Input validation** - Handle invalid input gracefully
2. **Error handling** - Provide helpful error messages
3. **Score tracking** - Track attempts and calculate scores
4. **Replay functionality** - Allow multiple games
5. **Statistics** - Show game statistics
6. **Difficulty levels** - Different number ranges

---

## 📋 Learning Objectives Review

This project will reinforce:
- **Variables**: Mutable game state, immutable configuration
- **Functions**: Modular code organization
- **Control Flow**: Game loops, conditional logic
- **Ownership**: String handling, data movement
- **Borrowing**: Efficient data access
- **Pattern Matching**: Input parsing and game state

---

## 🏗️ Project Structure

We'll build the game in stages:
1. Basic game logic
2. Input validation and error handling
3. Score tracking system
4. Replay functionality
5. Statistics and enhancements

---

## 🔧 Stage 1: Basic Game Logic

Let's start with the core game mechanics:

In [ ]:
use std::io;
use std::cmp::Ordering;

// Game configuration
struct GameConfig {
    min_number: u32,
    max_number: u32,
    max_attempts: u32,
}

impl GameConfig {
    fn new(min: u32, max: u32, attempts: u32) -> Self {
        GameConfig {
            min_number: min,
            max_number: max,
            max_attempts: attempts,
        }
    }
    
    fn easy() -> Self {
        Self::new(1, 50, 10)
    }
    
    fn medium() -> Self {
        Self::new(1, 100, 8)
    }
    
    fn hard() -> Self {
        Self::new(1, 200, 6)
    }
}

// Generate a random number (simplified for Jupyter)
fn generate_secret_number(config: &GameConfig) -> u32 {
    // In a real implementation, you'd use rand crate
    // For this demo, we'll use a simple calculation
    use std::collections::hash_map::DefaultHasher;
    use std::hash::{Hash, Hasher};
    
    let mut hasher = DefaultHasher::new();
    std::time::SystemTime::now().hash(&mut hasher);
    let hash = hasher.finish();
    
    config.min_number + (hash as u32) % (config.max_number - config.min_number + 1)
}

fn basic_game_demo() {
    let config = GameConfig::easy();
    let secret = generate_secret_number(&config);
    
    println!("🎮 Number Guessing Game - Basic Version");
    println!("I'm thinking of a number between {} and {}", config.min_number, config.max_number);
    println!("Secret number (for demo): {}", secret);  // Remove this in real game!
    
    // Simulate a few guesses
    let test_guesses = [25, 40, secret];
    
    for (attempt, &guess) in test_guesses.iter().enumerate() {
        println!("\nAttempt {}: Guessing {}", attempt + 1, guess);
        
        match guess.cmp(&secret) {
            Ordering::Less => println!("📈 Too small!"),
            Ordering::Greater => println!("📉 Too big!"),
            Ordering::Equal => {
                println!("🎉 You win! The number was {}", secret);
                break;
            }
        }
    }
}

basic_game_demo();

## 🔧 Stage 2: Input Validation and Error Handling

Now let's add robust input handling:

In [ ]:
#[derive(Debug)]
enum InputError {
    InvalidNumber,
    OutOfRange,
    Empty,
}

impl std::fmt::Display for InputError {
    fn fmt(&self, f: &mut std::fmt::Formatter) -> std::fmt::Result {
        match self {
            InputError::InvalidNumber => write!(f, "Please enter a valid number"),
            InputError::OutOfRange => write!(f, "Number is out of range"),
            InputError::Empty => write!(f, "Please enter something"),
        }
    }
}

fn parse_guess(input: &str, config: &GameConfig) -> Result<u32, InputError> {
    // Check for empty input
    let trimmed = input.trim();
    if trimmed.is_empty() {
        return Err(InputError::Empty);
    }
    
    // Try to parse as number
    let number: u32 = trimmed.parse()
        .map_err(|_| InputError::InvalidNumber)?;
    
    // Check range
    if number < config.min_number || number > config.max_number {
        return Err(InputError::OutOfRange);
    }
    
    Ok(number)
}

fn input_validation_demo() {
    let config = GameConfig::easy();
    
    println!("🔍 Input Validation Demo");
    println!("Valid range: {} to {}", config.min_number, config.max_number);
    
    // Test various inputs
    let test_inputs = [
        "25",      // Valid
        "abc",     // Invalid number
        "",        // Empty
        "  ",      // Whitespace only
        "0",       // Out of range (too low)
        "100",     // Out of range (too high for easy mode)
        "  42  ",  // Valid with whitespace
    ];
    
    for input in &test_inputs {
        print!("Input: '{}' -> ", input);
        match parse_guess(input, &config) {
            Ok(number) => println!("✅ Valid: {}", number),
            Err(error) => println!("❌ Error: {}", error),
        }
    }
}

input_validation_demo();

## 🔧 Stage 3: Score Tracking System

Let's add a scoring system to track player performance:

In [ ]:
#[derive(Debug, Clone)]
struct GameResult {
    won: bool,
    attempts_used: u32,
    max_attempts: u32,
    difficulty: String,
    score: u32,
}

impl GameResult {
    fn new(won: bool, attempts: u32, config: &GameConfig, difficulty: &str) -> Self {
        let score = if won {
            // Higher score for fewer attempts and harder difficulty
            let base_score = (config.max_attempts - attempts + 1) * 10;
            let difficulty_multiplier = match difficulty {
                "Easy" => 1,
                "Medium" => 2,
                "Hard" => 3,
                _ => 1,
            };
            base_score * difficulty_multiplier
        } else {
            0
        };
        
        GameResult {
            won,
            attempts_used: attempts,
            max_attempts: config.max_attempts,
            difficulty: difficulty.to_string(),
            score,
        }
    }
}

struct GameStats {
    games_played: u32,
    games_won: u32,
    total_score: u32,
    best_score: u32,
    results: Vec<GameResult>,
}

impl GameStats {
    fn new() -> Self {
        GameStats {
            games_played: 0,
            games_won: 0,
            total_score: 0,
            best_score: 0,
            results: Vec::new(),
        }
    }
    
    fn add_result(&mut self, result: GameResult) {
        self.games_played += 1;
        if result.won {
            self.games_won += 1;
        }
        self.total_score += result.score;
        if result.score > self.best_score {
            self.best_score = result.score;
        }
        self.results.push(result);
    }
    
    fn win_rate(&self) -> f64 {
        if self.games_played == 0 {
            0.0
        } else {
            (self.games_won as f64 / self.games_played as f64) * 100.0
        }
    }
    
    fn average_score(&self) -> f64 {
        if self.games_played == 0 {
            0.0
        } else {
            self.total_score as f64 / self.games_played as f64
        }
    }
    
    fn display_stats(&self) {
        println!("\n📊 Game Statistics");
        println!("═══════════════════");
        println!("Games played: {}", self.games_played);
        println!("Games won: {}", self.games_won);
        println!("Win rate: {:.1}%", self.win_rate());
        println!("Total score: {}", self.total_score);
        println!("Best score: {}", self.best_score);
        println!("Average score: {:.1}", self.average_score());
        
        if !self.results.is_empty() {
            println!("\n📈 Recent Games:");
            for (i, result) in self.results.iter().rev().take(5).enumerate() {
                let status = if result.won { "Won" } else { "Lost" };
                println!("  {}. {} - {} attempts, {} points ({})", 
                    self.results.len() - i, 
                    status, 
                    result.attempts_used, 
                    result.score,
                    result.difficulty
                );
            }
        }
    }
}

fn scoring_system_demo() {
    let mut stats = GameStats::new();
    
    // Simulate some games
    let config_easy = GameConfig::easy();
    let config_medium = GameConfig::medium();
    
    // Game 1: Easy win in 3 attempts
    let result1 = GameResult::new(true, 3, &config_easy, "Easy");
    stats.add_result(result1);
    
    // Game 2: Medium win in 5 attempts
    let result2 = GameResult::new(true, 5, &config_medium, "Medium");
    stats.add_result(result2);
    
    // Game 3: Easy loss
    let result3 = GameResult::new(false, config_easy.max_attempts, &config_easy, "Easy");
    stats.add_result(result3);
    
    // Game 4: Medium win in 2 attempts (high score!)
    let result4 = GameResult::new(true, 2, &config_medium, "Medium");
    stats.add_result(result4);
    
    stats.display_stats();
}

scoring_system_demo();

## 🔧 Stage 4: Complete Game Implementation

Now let's put it all together in a complete game:

In [ ]:
struct NumberGuessingGame {
    stats: GameStats,
}

impl NumberGuessingGame {
    fn new() -> Self {
        NumberGuessingGame {
            stats: GameStats::new(),
        }
    }
    
    fn choose_difficulty(&self) -> GameConfig {
        println!("\n🎯 Choose difficulty:");
        println!("1. Easy (1-50, 10 attempts)");
        println!("2. Medium (1-100, 8 attempts)");
        println!("3. Hard (1-200, 6 attempts)");
        
        // For demo, we'll simulate choosing medium
        println!("Choosing Medium difficulty for demo...");
        GameConfig::medium()
    }
    
    fn play_single_game(&mut self, config: &GameConfig, difficulty_name: &str) -> GameResult {
        let secret_number = generate_secret_number(config);
        let mut attempts = 0;
        
        println!("\n🎮 New Game Started!");
        println!("I'm thinking of a number between {} and {}", 
                config.min_number, config.max_number);
        println!("You have {} attempts.", config.max_attempts);
        println!("Secret number (for demo): {}", secret_number);
        
        // Simulate some guesses for demo
        let demo_guesses = [
            ("50", "Valid guess"),
            ("75", "Valid guess"),
            ("abc", "Invalid input"),
            ("62", "Valid guess"),
            (&secret_number.to_string(), "Winning guess"),
        ];
        
        for (input, description) in &demo_guesses {
            if attempts >= config.max_attempts {
                break;
            }
            
            println!("\n📝 Simulating input: '{}' ({})", input, description);
            
            match parse_guess(input, config) {
                Ok(guess) => {
                    attempts += 1;
                    println!("Attempt {}/{}: You guessed {}", attempts, config.max_attempts, guess);
                    
                    match guess.cmp(&secret_number) {
                        Ordering::Less => {
                            println!("📈 Too small! Try a larger number.");
                            if attempts < config.max_attempts {
                                let remaining = config.max_attempts - attempts;
                                println!("You have {} attempts remaining.", remaining);
                            }
                        },
                        Ordering::Greater => {
                            println!("📉 Too big! Try a smaller number.");
                            if attempts < config.max_attempts {
                                let remaining = config.max_attempts - attempts;
                                println!("You have {} attempts remaining.", remaining);
                            }
                        },
                        Ordering::Equal => {
                            println!("🎉 Congratulations! You guessed the number!");
                            let result = GameResult::new(true, attempts, config, difficulty_name);
                            println!("🏆 You earned {} points!", result.score);
                            return result;
                        }
                    }
                },
                Err(error) => {
                    println!("❌ {}", error);
                    println!("Please enter a number between {} and {}", 
                            config.min_number, config.max_number);
                }
            }
        }
        
        // If we get here, the player ran out of attempts
        println!("\n💔 Game Over! You've used all {} attempts.", config.max_attempts);
        println!("The secret number was: {}", secret_number);
        
        GameResult::new(false, attempts, config, difficulty_name)
    }
    
    fn run(&mut self) {
        println!("🎮 Welcome to the Enhanced Number Guessing Game!");
        println!("═══════════════════════════════════════════════");
        
        // Play a few demo games
        for game_num in 1..=2 {
            println!("\n🎯 Starting Game {}", game_num);
            
            let config = self.choose_difficulty();
            let difficulty_name = "Medium";  // For demo
            
            let result = self.play_single_game(&config, difficulty_name);
            self.stats.add_result(result);
            
            println!("\n" + "─".repeat(50).as_str());
        }
        
        // Show final statistics
        self.stats.display_stats();
        
        println!("\n🙏 Thanks for playing!");
    }
}

fn complete_game_demo() {
    let mut game = NumberGuessingGame::new();
    game.run();
}

complete_game_demo();

---

## 🎯 Your Turn: Enhancements

Now it's time to enhance the game further! Try implementing these features:

In [ ]:
// TODO: Implement these enhancements

// Enhancement 1: Hint System
fn provide_hint(secret: u32, guess: u32, attempt: u32) -> String {
    // TODO: Provide helpful hints based on how close the guess is
    // Ideas:
    // - "Very close!" if within 5
    // - "Getting warmer" if within 10
    // - "Cold" if more than 20 away
    // - After 3 attempts, give range hints
    
    String::from("Implement hint logic here")
}

// Enhancement 2: Custom Difficulty
fn create_custom_difficulty() -> GameConfig {
    // TODO: Allow players to set their own range and attempt limits
    // Validate that min < max and attempts > 0
    
    GameConfig::new(1, 100, 10)  // Placeholder
}

// Enhancement 3: Achievement System
#[derive(Debug)]
enum Achievement {
    FirstWin,
    PerfectGame,    // Win in 1 attempt
    Efficient,      // Win in <= 3 attempts
    Persistent,     // Play 10 games
    HighScorer,     // Score over 100 points
}

fn check_achievements(stats: &GameStats, latest_result: &GameResult) -> Vec<Achievement> {
    // TODO: Check if any achievements were unlocked
    // Return a vector of newly earned achievements
    
    Vec::new()  // Placeholder
}

// Enhancement 4: Save/Load Statistics
fn save_stats_to_string(stats: &GameStats) -> String {
    // TODO: Serialize game statistics to a string format
    // You could use JSON-like format or simple key-value pairs
    
    String::from("Implement save logic here")
}

fn load_stats_from_string(data: &str) -> Result<GameStats, String> {
    // TODO: Deserialize game statistics from string
    // Handle parsing errors gracefully
    
    Err("Implement load logic here".to_string())
}

println!("🚀 Enhancement challenges ready for implementation!");
println!("Choose one or more features to add to your game.");

---

## 🧪 Self-Assessment Checklist

**Concepts Demonstrated** - Check off what you've successfully implemented:

### Core Rust Concepts
- [ ] **Variables & Mutability**: Used `mut` for game state, immutable for configuration
- [ ] **Data Types**: Worked with `u32`, `String`, `bool`, custom structs
- [ ] **Functions**: Created modular functions with parameters and return values
- [ ] **Control Flow**: Used `match`, `if/else`, loops for game logic
- [ ] **Ownership**: Managed `String` ownership in function calls
- [ ] **Borrowing**: Used references (`&`) to avoid unnecessary moves
- [ ] **Pattern Matching**: Used `match` for `Ordering` and `Result` types

### Programming Practices
- [ ] **Error Handling**: Created custom error types and used `Result`
- [ ] **Code Organization**: Separated concerns into different functions/structs
- [ ] **User Experience**: Provided clear feedback and instructions
- [ ] **Data Validation**: Validated user input before processing
- [ ] **State Management**: Tracked game state and statistics

### Advanced Features
- [ ] **Struct Implementation**: Created structs with associated functions
- [ ] **Vector Usage**: Stored game history in a `Vec`
- [ ] **String Manipulation**: Formatted output and processed input
- [ ] **Mathematical Operations**: Calculated scores and statistics

---

## 🤔 Reflection Questions

Take a moment to reflect on your learning:

1. **Which Rust concepts felt most natural to use in this project?**
2. **What challenges did you face with ownership and borrowing?**
3. **How did pattern matching with `match` improve your code?**
4. **What would you do differently if you started this project over?**
5. **How does Rust's error handling compare to other languages you know?**

Write your thoughts in the cell below:

**Your Reflections:**

1. 

2. 

3. 

4. 

5. 

---

## 🎓 Next Steps

Congratulations on completing the beginner capstone project! You've successfully:

✅ **Built a complete Rust application**  
✅ **Applied all beginner-level concepts**  
✅ **Handled user input and errors gracefully**  
✅ **Organized code into logical modules**  
✅ **Implemented a scoring and statistics system**  

### Ready for Intermediate Stage?

You're now prepared to tackle:
- **Structs and Methods** - More complex data modeling
- **Enums and Advanced Pattern Matching** - Sophisticated state management
- **Collections** - Working with `Vec`, `HashMap`, and more
- **Error Handling** - Advanced error types and propagation
- **Generics and Traits** - Code reuse and abstraction

### Portfolio Project

Consider expanding this game into a portfolio piece:
- Add a graphical interface
- Implement network multiplayer
- Create different game modes
- Add sound effects and animations
- Deploy as a web application

---

**🎉 Excellent work!** You've mastered the fundamentals of Rust programming and built something genuinely useful and fun!